In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document

def load_and_chunk_documents(file_path: str) -> List:
    """
    Загружает документ из файла и разбивает его на чанки.
    """
    loader = TextLoader(file_path, encoding='utf-8')
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1600,
        chunk_overlap=200,
        separators=[
            "\n# ", "\n## ", "\n- ", "\n\n", "\n", " ", ""
        ]
    )
    chunked_docs = text_splitter.split_documents(documents)
    print(f"Документ '{file_path}' разбит на {len(chunked_docs)} чанков.")
    return chunked_docs

In [ ]:
chunked_docs = load_and_chunk_documents("data/docs.md")

In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from typing import List
from langchain_core.documents import Document
import os
import shutil
from dotenv import load_dotenv

def create_vector_store(chunked_docs: List, collection_name: str = "credit_platform_docs"):
    """
    Создает векторное хранилище из чанков документов.
    """
    persist_dir = "chroma_db"
    if os.path.exists(persist_dir):
        shutil.rmtree(persist_dir)

    # Загрузка переменных окружения из .env
    load_dotenv()

    embeddings = OpenAIEmbeddings(
        model="text-embedding-3-small"
    )

    vector_store = Chroma.from_documents(
        documents=chunked_docs,
        embedding=embeddings,
        collection_name=collection_name,
        persist_directory=persist_dir
    )

    print(f"Векторное хранилище '{collection_name}' создано и содержит {len(vector_store.get()['ids'])} документов.")
    return vector_store

In [ ]:
# Файл: tools/qa_tool.py
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
from langchain_chroma import Chroma


# Шаблон промпта, который инструктирует модель
RAG_PROMPT_TEMPLATE = """
Ты — дружелюбный ассистент, отвечающий по вопросам взаимного кредитования. 
Используй предоставленный ниже контекст, чтобы ответить на вопрос пользователя. Ответь коротко и по существу, дружелюбным тоном. 
Не пересказывай весь документ, а приведи только ту информацию, что отвечает на вопрос пользователя. 
Не добавляй информацию вне предоставленного контекста.
Не включай личные мнения, только факты
Не повторяй вопрос и не перечисляй пункты, если об этом не просили. 

Контекст:
{context}

Вопрос:
{question}

Ответ:
"""


def format_docs(docs):
    """
    Форматирует список документов в единую строку.
    """
    return "\n\n".join(doc.page_content for doc in docs)

def create_qa_rag_chain(vector_store: Chroma):
    # Загрузка переменных окружения из .env
    load_dotenv()
    openai_api_key = os.getenv("OPENAI_API_KEY")

    retriever = vector_store.as_retriever(search_kwargs={"k": 2})

    llm = ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0.2,
        openai_api_key=openai_api_key
    )
    prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    return rag_chain

In [ ]:
def create_p2p_agent():
    """
    Создает и настраивает агента для P2P платформы.
    """
    # --- Инициализация инструментов ---

    # Инструмент для ответов на вопросы (RAG)
    print("Инициализация RAG-инструмента...")
    
    vector_store = create_vector_store(chunked_docs, "p2p_agent_db")
    agent_executor = create_qa_rag_chain(vector_store)
    
    print("Агент успешно создан.")
    return agent_executor


In [ ]:
p2p_agent_executor = create_p2p_agent()

print("\n--- Тест 1 ---")
response1 = p2p_agent_executor.invoke("Какие у вас гарантии безопасности?")
print("Ответ интеллектуального агента:", response1)



# print("\n--- Тест 3 ---")
# response3 = p2p_agent_executor.invoke("Какой минимальный срок для вклада?")
# print("Ответ интеллектуального агента:", response3)

Инициализация RAG-инструмента...
Документ 'data/docs.md' разбит на 10 чанков.
Векторное хранилище 'p2p_agent_db' создано и содержит 10 документов.
Агент успешно создан.

--- Тест 1 ---
Ответ интеллектуального агента: У нас есть несколько мер безопасности, которые обеспечивают надежность платформы. Во-первых, смарт-контракты платформы аудируются сторонними компаниями на предмет уязвимостей. Во-вторых, мы не храним ваши пароли или приватные ключи, и ваша криптовалюта хранится под вашим контролем в смарт-контракте. Кроме того, принцип over-collateralization обеспечивает надежное покрытие вашей долговой нагрузки залогом. Все транзакции прозрачны и записываются в блокчейне, что исключает скрытые комиссии или изменения условий без уведомления.


In [6]:
print("\n--- Тест 2 ---")
response2 = p2p_agent_executor.invoke("Хочу вложить 3000eth на 3 месяца и на 1000usdt")
print("Ответ интеллектуального агента:", response2)


--- Тест 2 ---
Ответ интеллектуального агента: Из предоставленного контекста не ясно, предоставляется ли возможность вложить ETH на платформе DeFi Credit. Поэтому я не могу точно сказать, можно ли вложить 3000 ETH на 3 месяца и на 1000 USDT на данной платформе. Рекомендуется обратиться к официальным источникам или поддержке платформы для получения более точной информации.
